# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

citiData = pd.read_csv("../output/cities.csv")
citiData.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Grindavik,63.84,-22.43,51.80,87,75,13.87,IS,1594475370
1,1,Mataura,-46.19,168.86,44.20,89,100,4.81,NZ,1594475370
2,2,Dikson,73.51,80.55,43.25,86,68,13.15,RU,1594475370
3,3,Bluff,-46.60,168.33,49.86,78,100,25.03,NZ,1594475370
4,4,Busselton,-33.65,115.33,46.00,67,0,11.52,AU,1594475370


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = citiData[["Lat", "Lng"]]
humidity = citiData["Humidity"]

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=85,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weatherData = citiData.loc[(citiData["Max Temp"] > 70) & (citiData["Max Temp"] < 80) & (citiData["Wind Speed"] < 10) & (citiData["Cloudiness"] == 0)]

weatherData = weatherData.dropna()

weatherData


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
111,111,Nizhniy Kuranakh,58.84,125.49,72.43,66,0,3.71,RU,1594475376
272,272,Mandalgovi,45.76,106.27,70.57,34,0,8.39,MN,1594475387
292,292,Krasnovishersk,60.41,57.08,75.11,41,0,8.55,RU,1594475388
329,329,Marquette,46.53,-87.63,73.00,76,0,6.53,US,1594475391
393,393,Verkhnevilyuysk,63.45,120.32,74.84,65,0,7.74,RU,1594475394
435,435,Cuencamé de Ceniceros,24.87,-103.70,75.33,40,0,9.22,MX,1594475397
445,445,Uva,56.99,52.19,78.01,50,0,1.01,RU,1594475398


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weatherData
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
111,111,Nizhniy Kuranakh,58.84,125.49,72.43,66,0,3.71,RU,1594475376,
272,272,Mandalgovi,45.76,106.27,70.57,34,0,8.39,MN,1594475387,
292,292,Krasnovishersk,60.41,57.08,75.11,41,0,8.55,RU,1594475388,
329,329,Marquette,46.53,-87.63,73.00,76,0,6.53,US,1594475391,
393,393,Verkhnevilyuysk,63.45,120.32,74.84,65,0,7.74,RU,1594475394,
435,435,Cuencamé de Ceniceros,24.87,-103.70,75.33,40,0,9.22,MX,1594475397,
445,445,Uva,56.99,52.19,78.01,50,0,1.01,RU,1594475398,


In [7]:

# geocoordinates

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {   
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    citi_lat = row["Lat"]
    citi_lng = row["Lng"]
    
    params["location"] = f"{citi_lat},{citi_lng}"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params = params).json()
    
    try:
        
        print(f"Closest hotel is {response['results'][0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
    except:
        
        print("Missing field/result... skipping.")


Retrieving Results for Index 111: Nizhniy Kuranakh.
Closest hotel is Park Pobedy.
Retrieving Results for Index 272: Mandalgovi.
Closest hotel is В.Ганбат.
Retrieving Results for Index 292: Krasnovishersk.
Closest hotel is Inn Serebryaniy rodnik.
Retrieving Results for Index 329: Marquette.
Closest hotel is Al Quaal Lodge.
Retrieving Results for Index 393: Verkhnevilyuysk.
Closest hotel is Gostinitsa Mila.
Retrieving Results for Index 435: Cuencamé de Ceniceros.
Closest hotel is Casa De Mary.
Retrieving Results for Index 445: Uva.
Closest hotel is B&B Zhemchuzhina.


In [8]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
111,111,Nizhniy Kuranakh,58.84,125.49,72.43,66,0,3.71,RU,1594475376,Park Pobedy
272,272,Mandalgovi,45.76,106.27,70.57,34,0,8.39,MN,1594475387,В.Ганбат
292,292,Krasnovishersk,60.41,57.08,75.11,41,0,8.55,RU,1594475388,Inn Serebryaniy rodnik
329,329,Marquette,46.53,-87.63,73.00,76,0,6.53,US,1594475391,Al Quaal Lodge
393,393,Verkhnevilyuysk,63.45,120.32,74.84,65,0,7.74,RU,1594475394,Gostinitsa Mila
435,435,Cuencamé de Ceniceros,24.87,-103.70,75.33,40,0,9.22,MX,1594475397,Casa De Mary
445,445,Uva,56.99,52.19,78.01,50,0,1.01,RU,1594475398,B&B Zhemchuzhina


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))